# 🤗 demo

## Pipelines

In [ ]:
# Note: not tested on Colab, just installation suggestions
# !pip install -qq transformers diffusers ffmpeg accelerate

In [ ]:
from transformers import pipeline

In [ ]:
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-6-6", device="cuda")

In [ ]:
text = """The Prague Spring (Czech: Pražské jaro, Slovak: Pražská jar) was a period of political liberalization and mass protest in the Czechoslovak Socialist Republic. It began on 5 January 1968, when reformist Alexander Dubček was elected First Secretary of the Communist Party of Czechoslovakia (KSČ), and continued until 21 August 1968, when the Soviet Union and most of Warsaw Pact members invaded the country to suppress the reforms.

The Prague Spring reforms were a strong attempt by Dubček to grant additional rights to the citizens of Czechoslovakia in an act of partial decentralization of the economy and democratization. The freedoms granted included a loosening of restrictions on the media, speech and travel. After national discussion of dividing the country into a federation of three republics, Bohemia, Moravia-Silesia and Slovakia, Dubček oversaw the decision to split into two, the Czech Socialist Republic and Slovak Socialist Republic.[1] This dual federation was the only formal change that survived the invasion.

The reforms, especially the decentralization of administrative authority, were not received well by the Soviets, who, after failed negotiations, sent half a million Warsaw Pact troops and tanks to occupy the country. The New York Times cited reports of 650,000 men equipped with the most modern and sophisticated weapons in the Soviet military catalogue.[2] A massive wave of emigration swept the nation. Resistance was mounted throughout the country, involving attempted fraternization, sabotage of street signs, defiance of curfews, etc. While the Soviet military had predicted that it would take four days to subdue the country, the resistance held out for eight months until diplomatic maneuvers finally circumvented it. It became a high-profile example of civilian-based defense; there were sporadic acts of violence and several protest suicides by self-immolation (the most famous being that of Jan Palach), but no military resistance. Czechoslovakia remained controlled by the Soviet Union until 1989, when the Velvet Revolution peacefully ended the communist regime; the last Soviet troops left the country in 1991.

After the invasion, Czechoslovakia entered a period known as normalization (Czech: normalizace, Slovak: normalizácia), in which new leaders attempted to restore the political and economic values that had prevailed before Dubček gained control of the KSČ. Gustáv Husák, who replaced Dubček as First Secretary and also became President, reversed almost all of the reforms. The Prague Spring inspired music and literature including the work of Václav Havel, Karel Husa, Karel Kryl and Milan Kundera's novel The Unbearable Lightness of Being."""

In [ ]:
summarizer(text)

In [ ]:
pipe = pipeline("text-classification", device="cuda")

In [ ]:
pipe("This restaurant is awesome")

In [ ]:
pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment")

In [ ]:
pipe("This restaurant is awesome")

In [ ]:
pipe2 = pipeline("text-classification", model="michellejieli/emotion_text_classifier")

In [ ]:
pipe2('Phew! This is awesome shit!')

In [ ]:
pipe2(text[:2000])

## Tokenizers and models

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
tokenizer("Using a Transformer network is simple")

In [ ]:
tokenizer.encode("Using a Transformer network is simple")

In [ ]:
tokenizer.decode(_)

In [ ]:
model_inputs = tokenizer("Using a Transformer network is rather boring and complicated", padding=True, return_tensors="pt")

model_inputs

In [ ]:
output = model(**model_inputs)

output

In [ ]:
from scipy.special import softmax

def is_promoter(model_outputs, categories):
    logits = model_outputs['logits'].detach().numpy()
    probs = softmax(logits, axis=1)[0]
    probs = map(float, probs)

    return dict(zip(categories, probs))

In [ ]:
is_promoter(output, ['negative', 'postitive'])

In [ ]:
import numpy as np

logits = output['logits'].detach().numpy()[0,:]
np.exp(logits) / (np.sum(np.exp(logits)))

## More demo

### Stable Diffusion (Text-to-image)

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

In [ ]:
from huggingface_hub import notebook_login
# from huggingface_hub.hf_api import HfFolder; HfFolder.save_token('YOUR_HF_TOKEN_HERE')

notebook_login()

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

In [ ]:
%%time
prompt = "painted portrait of rugged zeus, god of thunder, greek god, white hair, masculine, mature, handsome, upper body, muscular, hairy torso, fantasy, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by gaston bussiere and alphonse mucha"
image = pipe(prompt).images[0]

In [ ]:
image

## Whisper2

In [ ]:
from transformers import pipeline

MODEL_NAME = "openai/whisper-large-v2"
device = 0

pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    device=device
)

In [ ]:
text = pipe("5522883-AManCalledOve.mp3", return_timestamps=True)["text"]

text